 ## GCN

In [1]:
!pip install torch torchvision torchaudio
!pip install geoopt
!pip install networkx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 4.6 MB/s eta 0:00:00


In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
from torch import optim
import geoopt
from geoopt import ManifoldParameter
import os
from tqdm import tqdm
import pandas as pd
from scipy.sparse import csr_matrix, diags
import numpy as np
import random
from sklearn.model_selection import train_test_split
import scipy.sparse as sp

##############################################
# Data Loading and Preprocessing
##############################################

def load_synthetic_data(dataset_str, use_feats, data_path):
    object_to_idx = {}
    idx_counter = 0
    edges = []

    # Read all edges from the CSV file
    with open(os.path.join(data_path, f"{dataset_str}.edges.csv"), 'r') as f:
        all_edges = f.readlines()

    for line in all_edges:
        n1, n2 = line.rstrip().split(',')

        # Assign unique indices to nodes
        if n1 in object_to_idx:
            i = object_to_idx[n1]
        else:
            i = idx_counter
            object_to_idx[n1] = i
            idx_counter += 1

        if n2 in object_to_idx:
            j = object_to_idx[n2]
        else:
            j = idx_counter
            object_to_idx[n2] = j
            idx_counter += 1

        edges.append((i, j))

    num_nodes = len(object_to_idx)

    # Initialize adjacency matrix using LIL format for efficient assignment
    adj = sp.lil_matrix((num_nodes, num_nodes), dtype=np.float32)

    for i, j in edges:
        adj[i, j] = 1.0  # Undirected graph: set both (i, j) and (j, i)
        adj[j, i] = 1.0

    # Convert adjacency matrix to CSR format for efficient arithmetic operations
    adj = adj.tocsr()

    # Load features
    if use_feats:
        features = sp.load_npz(os.path.join(data_path, f"{dataset_str}.feats.npz")).astype(np.float32)
    else:
        features = sp.eye(adj.shape[0], dtype=np.float32)

    # Load labels
    labels = np.load(os.path.join(data_path, f"{dataset_str}.labels.npy"))

    return adj, features, labels, object_to_idx


# Define file paths
data_path = ''  # Update this to your actual data path
dataset_str = 'disease_lp'

# Load data
adj, features, labels, object_to_idx = load_synthetic_data(dataset_str, use_feats=True, data_path=data_path)

num_nodes = adj.shape[0]
num_features = features.shape[1]
num_classes = len(np.unique(labels))  # Assuming labels are integer-encoded

print(f"Number of Nodes: {num_nodes}")
print(f"Number of Features: {num_features}")
print(f"Number of Classes: {num_classes}")

# Convert adjacency matrix to a sparse matrix and add self-loops
adj = adj + diags([1e-5] * num_nodes)

# Compute degree
degrees = np.array(adj.sum(axis=1)).flatten()

# Compute D^-0.5 (inverse square root of degree matrix)
inv_sqrt_deg = np.power(degrees, -0.5)
inv_sqrt_deg[np.isinf(inv_sqrt_deg)] = 0
D_inv_sqrt = diags(inv_sqrt_deg)

# Normalize adjacency matrix: D^-0.5 * A * D^-0.5
norm_adj = D_inv_sqrt @ adj @ D_inv_sqrt

# Convert back to a PyTorch tensor
norm_adj = torch.tensor(norm_adj.toarray(), dtype=torch.float32)

# Node Features: Load from the features matrix
features = csr_matrix(features)
features = torch.tensor(features.toarray(), dtype=torch.float32)

# Move tensors to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
features, norm_adj = features.to(device), norm_adj.to(device)

print(f"Using device: {device}")


###################################################################################
# GCN DEFINITION
###################################################################################

# Define the GCN Baseline Model
class GCNBaseline(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_relations):
        super(GCNBaseline, self).__init__()
        self.conv1 = nn.Linear(input_dim, hidden_dim)
        self.conv2 = nn.Linear(hidden_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim * 2, num_relations)  # For link prediction

    def forward(self, features, adj):
        x = F.relu(self.conv1(features))
        x = torch.matmul(adj, x)  # Graph convolution using adjacency matrix
        x = F.relu(self.conv2(x))
        return x

    def classify(self, head_idx, tail_idx, node_emb):
        head_emb = node_emb[head_idx]
        tail_emb = node_emb[tail_idx]
        score = self.classifier(torch.cat([head_emb, tail_emb], dim=1))
        return score

# Training function
def train_epoch(model, optimizer, criterion, heads, tails, relations, node_emb):
    model.train()
    optimizer.zero_grad()

    # Get predictions for head-tail pairs
    scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]

    # Compute loss
    loss = criterion(scores, relations)

    # Backpropagation
    loss.backward()
    optimizer.step()

    return loss.item()

# Link Prediction Model for Hyperbolic Graph Convolution
class LinkPredictionModel(nn.Module):
    def __init__(self, num_features, hidden_dim, num_relations):
        super(LinkPredictionModel, self).__init__()
        self.gcn = GCNBaseline(num_features, hidden_dim, num_relations)
        self.num_relations = num_relations
        self.relation_embeddings = nn.Embedding(num_relations, hidden_dim)  # Added this line

    def forward(self, features, adj):
        # Generate node embeddings
        node_embeddings = self.gcn(features, adj)
        return node_embeddings

    def classify(self, head_idx, tail_idx, node_embeddings):
        # Calculate scores for all relations
        head_emb = node_embeddings[head_idx]
        tail_emb = node_embeddings[tail_idx]

        # Use relation embeddings for the classification step
        relation_emb = self.relation_embeddings.weight  # Get the learned relation embeddings
        scores = torch.matmul(head_emb, relation_emb.T) + torch.matmul(tail_emb, relation_emb.T)
        return scores


############################################################
#  TRAINING FUNCTIONS
############################################################

def prepare_data(triples):
    """
    Prepare tensors for head, tail, and relation indices.
    """
    triples = torch.tensor(triples, dtype=torch.long, device=device)
    heads = triples[:, 0]
    relations = triples[:, 1]
    tails = triples[:, 2]
    return heads, tails, relations

def train_epoch(model, optimizer, criterion, heads, tails, relations, node_emb):
    model.train()
    optimizer.zero_grad()

    # Get relation scores
    scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]

    # Compute loss
    loss = criterion(scores, relations)
    loss.backward()
    optimizer.step()

    return loss.item()

def evaluate(model, criterion, heads, tails, relations, node_emb):
    model.eval()
    with torch.no_grad():
        scores = model.classify(heads, tails, node_emb)  # [batch_size, num_relations]
        loss = criterion(scores, relations).item()
        preds = torch.argmax(scores, dim=1)
        correct = (preds == relations).sum().item()
        total = relations.size(0)
        accuracy = correct / total
    return loss, accuracy

def generate_negative_edges(adj_matrix, num_neg_samples, excluded_edges=set()):
    neg_edges = set()
    while len(neg_edges) < num_neg_samples:
        i = random.randint(0, num_nodes - 1)
        j = random.randint(0, num_nodes - 1)
        if i == j:
            continue
        if adj_matrix[i, j] == 0 and (i, j) not in excluded_edges and (j, i) not in excluded_edges:
            neg_edges.add((i, j))
    return list(neg_edges)

# Extract existing edges
existing_edges = set()
adj_coo = adj.tocoo()
for i, j in zip(adj_coo.row, adj_coo.col):
    if i < j:  # To avoid duplicates in undirected graph
        existing_edges.add((i, j))

# Number of positive and negative samples
num_positive = len(existing_edges)
num_negative = num_positive  # Balance the dataset

# Generate negative edges
negative_edges = generate_negative_edges(adj, num_negative, excluded_edges=existing_edges)

# Create labels: 1 for positive, 0 for negative
positive_labels = np.ones(num_positive)
negative_labels = np.zeros(num_negative)

# Combine positive and negative edges
all_edges = list(existing_edges) + negative_edges
all_labels = np.concatenate([positive_labels, negative_labels])

# Split into train, val, test
train_edges, temp_edges, train_labels, temp_labels = train_test_split(all_edges, all_labels, test_size=0.3, random_state=42, stratify=all_labels)
val_edges, test_edges, val_labels, test_labels = train_test_split(temp_edges, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels)

# Prepare training, validation, and test data
train_triples = [(h, r, t) for (h, t), r in zip(train_edges, train_labels)]
val_triples = [(h, r, t) for (h, t), r in zip(val_edges, val_labels)]
test_triples = [(h, r, t) for (h, t), r in zip(test_edges, test_labels)]

train_heads, train_tails, train_relations = prepare_data(train_triples)
val_heads, val_tails, val_relations = prepare_data(val_triples)
test_heads, test_tails, test_relations = prepare_data(test_triples)

# Initialize the model
hidden_dim = 128
num_relations = len(np.unique(train_labels))
model = LinkPredictionModel(num_features = num_features, hidden_dim=hidden_dim, num_relations=num_relations).to(device)

# Optimizer Choices:
riemannian_adam = geoopt.optim.RiemannianAdam(model.parameters(), lr=1e-4)
riemannian_sgd = geoopt.optim.RiemannianSGD(model.parameters(), lr=1e-2)

# SET OPTIMIZER:
optimizer = riemannian_adam

# Define loss function for multi-class classification
criterion = nn.CrossEntropyLoss()

# Training parameters
num_epochs = 1000
best_val_accuracy = 0.0
best_test_accuracy = 0.0

train_losses, val_losses, val_accuracies, test_accuracies = [], [], [], []

print("\nStarting Training...\n")
for epoch in tqdm(range(1, num_epochs + 1)):
    # Forward pass to get node embeddings
    node_emb = model(features, norm_adj)  # [num_nodes, hidden_dim]

    # Training
    train_loss = train_epoch(model, optimizer, criterion, train_heads, train_tails, train_relations, node_emb)

    # Evaluation on Validation Set
    val_loss, val_accuracy = evaluate(model, criterion, val_heads, val_tails, val_relations, node_emb)

    # Early Stopping and Best Model Tracking
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        # Evaluate on Test Set
        test_loss, test_accuracy = evaluate(model, criterion, test_heads, test_tails, test_relations, node_emb)
        best_test_accuracy = test_accuracy

    # Print Epoch Results
    if epoch % 10 == 0 or epoch == 1:
        print(f"Epoch {epoch:03d}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}, Val Acc={val_accuracy:.4f}, Best Test Acc={best_test_accuracy:.4f}")

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)
    test_accuracies.append(test_accuracy)

print("\nTraining Completed.")
print(f"Best Validation Accuracy: {best_val_accuracy:.4f}")
print(f"Corresponding Test Accuracy: {best_test_accuracy:.4f}")


Number of Nodes: 2665
Number of Features: 11
Number of Classes: 2
Using device: cpu

Starting Training...



  0%|          | 2/1000 [00:00<02:29,  6.65it/s]

Epoch 001: Train Loss=3.3203, Val Loss=3.2491, Val Acc=0.4556, Best Test Acc=0.4637


  1%|          | 11/1000 [00:01<02:25,  6.78it/s]

Epoch 010: Train Loss=0.9399, Val Loss=0.8925, Val Acc=0.4168, Best Test Acc=0.4637


...

Epoch 980: Train Loss=0.3286, Val Loss=0.4269, Val Acc=0.8511, Best Test Acc=0.8650


 99%|█████████▉| 991/1000 [01:40<00:00, 11.33it/s]

Epoch 990: Train Loss=0.3281, Val Loss=0.4269, Val Acc=0.8486, Best Test Acc=0.8650


100%|██████████| 1000/1000 [01:40<00:00,  9.91it/s]

Epoch 1000: Train Loss=0.3277, Val Loss=0.4269, Val Acc=0.8486, Best Test Acc=0.8650

Training Completed.
Best Validation Accuracy: 0.8511
Corresponding Test Accuracy: 0.8650
